In [42]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import time
import itertools
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import seaborn as sns
import scipy.stats as stats
from scipy.stats import gamma
from scipy.stats import ortho_group

import cupy as cp
import cupyx.scipy 

from Test.hsic_naive import IndpTest_naive
from Test.hsic_lkgau import IndpTest_LKGaussian
from Test.hsic_lklap import IndpTest_LKLaplace
from Test.hsic_lkwgau import IndpTest_LKWeightGaussian
from Test.hsic_lkwlap import IndpTest_LKWeightLaplace
from Test.hsic_lkselect import IndpTest_LKSelect_GauLap
from Test.hsic_kselect import IndpTest_KSelect
from Test.dime_gaussian import IndpTest_DIME

device = torch.device('cuda:0')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [66]:
def generate_ISA(n,d,sigma_normal,alpha):
    
    x = np.concatenate((np.random.normal(-1, sigma_normal, n//2), np.random.normal(1, sigma_normal, n//2)))
    y = np.concatenate((np.random.normal(-1, sigma_normal, n//2), np.random.normal(1, sigma_normal, n//2)))
    p = np.random.permutation(n)
    y_p = y[p]

    D = np.zeros([2,n])
    D[0,:] = x
    D[1,:] = y_p

    theta = np.pi/4*alpha
    c, s = np.cos(theta), np.sin(theta)
    R = np.array(((c, -s), (s, c)))

    D_R = R@D
    X_mix = D_R[0,:].reshape(-1,1)
    Y_mix = D_R[1,:].reshape(-1,1)

    X_z = np.random.randn(n,d-1)
    Y_z = np.random.randn(n,d-1)

    X_con = np.concatenate((X_mix,X_z), axis=1)
    Y_con = np.concatenate((Y_mix,Y_z), axis=1)

    m_x = ortho_group.rvs(dim=d)
    m_y = ortho_group.rvs(dim=d)

    X = (m_x@X_con.T).T
    Y = (m_y@Y_con.T).T
    
    return X,Y

def Sinusoid(x, y, w):
    return 1 + np.sin(w*x)*np.sin(w*y)

def Sinusoid_Generator(n,w):
    i = 0
    output = np.zeros([n,2])
    while i < n:
        U = np.random.rand(1)
        V = np.random.rand(2)
        x0 = -np.pi + V[0]*2*np.pi
        x1 = -np.pi + V[1]*2*np.pi
        if U < 1/2 * Sinusoid(x0,x1,w):
            output[i, 0] = x0
            output[i, 1] = x1
            i = i + 1
    return output

# n = 1000
# d = 3
def sinedependence(n,d,seed = 0):
    np.random.seed(seed)
    mean = np.zeros(d)
    cov = np.eye(d)
    X = np.random.multivariate_normal(mean, cov, n)
    Z = np.random.randn(n)
    Y = 20*np.sin(4*np.pi*(X[:,0]**2 + X[:,1]**2))+Z 
    return X,Y

# n = 1000
# d = 4
def GSign(n,d, seed):
    np.random.seed(seed)
    mean = np.zeros(d)
    cov = np.eye(d)
    X = np.random.multivariate_normal(mean, cov, n)
    sign_X = np.sign(X)
    Z = np.random.randn(n)
    Y = np.abs(Z)*np.prod(sign_X,1)
    return X,Y



In [65]:
n = 900
d = 3
test_num = 10
dime_perm = 5
result_test_correct = np.zeros([6,test_num])
seed = 0 
for it in range(test_num):
    X, Y = sinedependence(n, d, seed)
    Y = Y.reshape(-1,1)
    X_tensor, Y_tensor = torch.tensor(X, device=device), torch.tensor(Y,device=device)

    #test#
    
    dime_estimator = IndpTest_DIME( X_tensor, Y_tensor, dime_perm = 5, alpha = 1.0, isotropic = True)
    results_dime = dime_estimator.perform_test()
    result_test_correct[0, it] = float(results_dime['h0_rejected'])
    print('Dime alpha = 1.0, isotropic kernel')
    print('threshold: ',results_dime["thresh"].item(), 'DiME', results_dime["test_stat"].item(), 'null rejected:' ,results_dime["h0_rejected"].item())

    dime_estimator = IndpTest_DIME( X_tensor, Y_tensor, dime_perm = 5, alpha = 1.0, isotropic = False)
    results_dime = dime_estimator.perform_test()
    result_test_correct[1, it] = float(results_dime['h0_rejected'])
    print('Dime alpha = 1.0, non-isotropic kernel')
    print('threshold: ',results_dime["thresh"].item(), 'DiME', results_dime["test_stat"].item(), 'null rejected' ,results_dime["h0_rejected"].item())

    dime_estimator = IndpTest_DIME( X_tensor, Y_tensor, dime_perm = 5, alpha = 2.0, isotropic = True)
    results_dime = dime_estimator.perform_test()
    result_test_correct[2, it] = float(results_dime['h0_rejected'])
    print('Dime alpha = 2.0, isotropic kernel')
    print('threshold: ',results_dime["thresh"].item(), 'DiME', results_dime["test_stat"].item(), 'null rejected' ,results_dime["h0_rejected"].item())

    dime_estimator = IndpTest_DIME( X_tensor, Y_tensor, dime_perm = 5, alpha = 2.0, isotropic = False)
    results_dime = dime_estimator.perform_test()
    result_test_correct[3, it] = float(results_dime['h0_rejected'])
    print('Dime alpha = 2.0, non-isotropic kernel')
    print('threshold: ',results_dime["thresh"].item(), 'DiME', results_dime["test_stat"].item(), 'null rejected' ,results_dime["h0_rejected"].item())

    dime_estimator = IndpTest_DIME( X_tensor, Y_tensor, dime_perm = 5, alpha = 5.0, isotropic = True)
    results_dime = dime_estimator.perform_test()
    result_test_correct[4, it] = float(results_dime['h0_rejected'])
    print('Dime alpha = 5.0, isotropic kernel')
    print('threshold: ',results_dime["thresh"].item(), 'DiME', results_dime["test_stat"].item(), 'null rejected' ,results_dime["h0_rejected"].item())

    dime_estimator = IndpTest_DIME( X_tensor, Y_tensor, dime_perm = 5, alpha = 5.0, isotropic = False)
    results_dime = dime_estimator.perform_test()
    result_test_correct[5, it] = float(results_dime['h0_rejected'])
    print('Dime alpha = 5.0, non-isotropic kernel')
    print('threshold: ',results_dime["thresh"].item(), 'DiME', results_dime["test_stat"].item(), 'null rejected' ,results_dime["h0_rejected"].item())

    # hsic0 = IndpTest_naive(X_tensor, Y_tensor, alpha=0.05, n_permutation=100, kernel_type="Gaussian", null_gamma = True)
    # results_all0 = hsic0.perform_test()
    # result_test_correct[0, it] = float(results_all0['h0_rejected'])

    # hsic1 = IndpTest_LKGaussian(X_tensor, Y_tensor, device, alpha=0.05, n_permutation=100, null_gamma = True, split_ratio = 0.5)
    # results_all1 = hsic1.perform_test(debug = -1, if_grid_search = True)
    # result_test_correct[1, it] = float(results_all1['h0_rejected'])

    # hsic2 = IndpTest_LKWeightGaussian(X_tensor, Y_tensor, device, alpha=0.05, n_permutation=100, null_gamma = True, split_ratio = 0.5)
    # results_all2 = hsic2.perform_test(debug = -1, if_grid_search = True)
    # result_test_correct[2, it] = float(results_all2['h0_rejected'])

    seed += 1
    print(it+1, np.sum(result_test_correct,1)/(it+1))

Dime alpha = 1.0, isotropic kernel
threshold:  0.017213361643603875 DiME 0.015209022152858687 null rejected: False
Dime alpha = 1.0, non-isotropic kernel
threshold:  -0.01073093983320117 DiME 0.010928274670368943 null rejected True
Dime alpha = 2.0, isotropic kernel
threshold:  0.009591960616041284 DiME 0.005039522001708541 null rejected False
Dime alpha = 2.0, non-isotropic kernel
threshold:  -0.000613758889413063 DiME 0.024177353187318396 null rejected True
Dime alpha = 5.0, isotropic kernel
threshold:  -0.00868324925864794 DiME 0.01796828704322806 null rejected True
Dime alpha = 5.0, non-isotropic kernel
threshold:  0.010861815433630184 DiME 0.011202952121232457 null rejected True
1 [0. 1. 0. 1. 1. 1.]
Dime alpha = 1.0, isotropic kernel
threshold:  0.010370256348952722 DiME 0.009548361963767071 null rejected: False
Dime alpha = 1.0, non-isotropic kernel
threshold:  -0.0026395562265806305 DiME 0.012399631086414509 null rejected True
Dime alpha = 2.0, isotropic kernel
threshold:  0.00

In [74]:
sample_sizes = (300,600,900,1200)
n_samples = len(sample_sizes)
n_tests = 9
test_num = 5
result_test_correct = np.zeros([n_tests,n_samples, test_num])
print(result_test_correct.shape)

(9, 4, 5)


In [69]:
n = 400
d = 4
test_num = 10

result_test_correct = np.zeros([6,test_num])
seed = 0 
for it in range(test_num):
    
    X, Y = GSign(n,d, seed)
    Y = Y.reshape(-1,1)
    X_tensor, Y_tensor = torch.tensor(X, device=device), torch.tensor(Y,device=device)

    #test#
    
    dime_estimator = IndpTest_DIME( X_tensor, Y_tensor, dime_perm = 5, alpha = 1.0, isotropic = True)
    results_dime = dime_estimator.perform_test()
    result_test_correct[0, it] = float(results_dime['h0_rejected'])
    print('Dime alpha = 1.0, isotropic kernel')
    print('threshold: ',results_dime["thresh"].item(), 'DiME', results_dime["test_stat"].item(), 'null rejected:' ,results_dime["h0_rejected"].item())

    dime_estimator = IndpTest_DIME( X_tensor, Y_tensor, dime_perm = 5, alpha = 1.0, isotropic = False)
    results_dime = dime_estimator.perform_test()
    result_test_correct[1, it] = float(results_dime['h0_rejected'])
    print('Dime alpha = 1.0, non-isotropic kernel')
    print('threshold: ',results_dime["thresh"].item(), 'DiME', results_dime["test_stat"].item(), 'null rejected' ,results_dime["h0_rejected"].item())

    dime_estimator = IndpTest_DIME( X_tensor, Y_tensor, dime_perm = 5, alpha = 2.0, isotropic = True)
    results_dime = dime_estimator.perform_test()
    result_test_correct[2, it] = float(results_dime['h0_rejected'])
    print('Dime alpha = 2.0, isotropic kernel')
    print('threshold: ',results_dime["thresh"].item(), 'DiME', results_dime["test_stat"].item(), 'null rejected' ,results_dime["h0_rejected"].item())

    dime_estimator = IndpTest_DIME( X_tensor, Y_tensor, dime_perm = 5, alpha = 2.0, isotropic = False)
    results_dime = dime_estimator.perform_test()
    result_test_correct[3, it] = float(results_dime['h0_rejected'])
    print('Dime alpha = 2.0, non-isotropic kernel')
    print('threshold: ',results_dime["thresh"].item(), 'DiME', results_dime["test_stat"].item(), 'null rejected' ,results_dime["h0_rejected"].item())

    dime_estimator = IndpTest_DIME( X_tensor, Y_tensor, dime_perm = 5, alpha = 5.0, isotropic = True)
    results_dime = dime_estimator.perform_test()
    result_test_correct[4, it] = float(results_dime['h0_rejected'])
    print('Dime alpha = 5.0, isotropic kernel')
    print('threshold: ',results_dime["thresh"].item(), 'DiME', results_dime["test_stat"].item(), 'null rejected' ,results_dime["h0_rejected"].item())

    dime_estimator = IndpTest_DIME( X_tensor, Y_tensor, dime_perm = 5, alpha = 5.0, isotropic = False)
    results_dime = dime_estimator.perform_test()
    result_test_correct[5, it] = float(results_dime['h0_rejected'])
    print('Dime alpha = 5.0, non-isotropic kernel')
    print('threshold: ',results_dime["thresh"].item(), 'DiME', results_dime["test_stat"].item(), 'null rejected' ,results_dime["h0_rejected"].item())

    seed += 1
    print(it+1, np.sum(result_test_correct,1)/(it+1))

Dime alpha = 1.0, isotropic kernel
threshold:  0.0006708887962485477 DiME 0.04456921173755024 null rejected: True
Dime alpha = 1.0, non-isotropic kernel
threshold:  -0.013133080297636823 DiME 0.0419807713191549 null rejected True
Dime alpha = 2.0, isotropic kernel
threshold:  -0.008024516155371586 DiME 0.05947545009005495 null rejected True
Dime alpha = 2.0, non-isotropic kernel
threshold:  -0.0396674707743081 DiME 0.045070527291972695 null rejected True
Dime alpha = 5.0, isotropic kernel
threshold:  0.01995930874766305 DiME 0.01976986479027243 null rejected False
Dime alpha = 5.0, non-isotropic kernel
threshold:  -0.01708408034850173 DiME 0.032022819703616534 null rejected True
1 [1. 1. 1. 1. 0. 1.]
Dime alpha = 1.0, isotropic kernel
threshold:  -0.012372196667962854 DiME 0.014060895291043707 null rejected: True
Dime alpha = 1.0, non-isotropic kernel
threshold:  -0.00017334037643834677 DiME 0.00641922513429094 null rejected True
Dime alpha = 2.0, isotropic kernel
threshold:  0.0049494

In [6]:
# ica experiments #

test_num = 100
result_all_correct = []

# alpha_set = np.linspace(0.0, 1.0, num=10)
alpha_set = [0.7]

confident_level = 0.05

for alpha in alpha_set:
    result_test_correct = np.zeros([3,test_num])
    
    for it in range(test_num):
        n = 128
        d = 4
        sigma_normal = 0.1
        X, Y = generate_ISA(n,d,sigma_normal,alpha)
        X_tensor, Y_tensor = torch.tensor(X), torch.tensor(Y)
        
        #test#
        hsic0 = IndpTest_naive(X_tensor, Y_tensor, alpha=0.05, n_permutation=100, kernel_type="Gaussian", null_gamma = True)
        results_all0 = hsic0.perform_test()
        result_test_correct[0, it] = float(results_all0['h0_rejected'])

        hsic1 = IndpTest_LKGaussian(X_tensor, Y_tensor, device, alpha=0.05, n_permutation=100, null_gamma = True, split_ratio = 0.5)
        results_all1 = hsic1.perform_test(debug = -1, if_grid_search = True)
        result_test_correct[1, it] = float(results_all1['h0_rejected'])

        hsic2 = IndpTest_LKWeightGaussian(X_tensor, Y_tensor, device, alpha=0.05, n_permutation=100, null_gamma = True, split_ratio = 0.5)
        results_all2 = hsic2.perform_test(debug = -1, if_grid_search = True)
        result_test_correct[2, it] = float(results_all2['h0_rejected'])

        print(alpha, it, np.sum(result_test_correct,1)/(it+1))
    
    result_all_correct.append(result_test_correct)

0.7 0 [1. 1. 1.]
0.7 1 [0.5 0.5 1. ]
0.7 2 [0.33333333 0.33333333 1.        ]
0.7 3 [0.25 0.5  1.  ]
0.7 4 [0.2 0.4 0.8]
0.7 5 [0.16666667 0.33333333 0.83333333]
0.7 6 [0.14285714 0.28571429 0.85714286]
0.7 7 [0.125 0.25  0.875]
0.7 8 [0.11111111 0.22222222 0.88888889]
0.7 9 [0.1 0.3 0.9]
0.7 10 [0.09090909 0.27272727 0.90909091]
0.7 11 [0.08333333 0.25       0.91666667]
0.7 12 [0.07692308 0.30769231 0.92307692]
0.7 13 [0.14285714 0.35714286 0.92857143]
0.7 14 [0.13333333 0.33333333 0.93333333]
0.7 15 [0.125  0.3125 0.875 ]
0.7 16 [0.11764706 0.29411765 0.88235294]
0.7 17 [0.11111111 0.27777778 0.88888889]
0.7 18 [0.10526316 0.31578947 0.89473684]
0.7 19 [0.1 0.3 0.9]
0.7 20 [0.0952381  0.33333333 0.9047619 ]
0.7 21 [0.09090909 0.36363636 0.90909091]
0.7 22 [0.08695652 0.34782609 0.91304348]
0.7 23 [0.08333333 0.375      0.91666667]
0.7 24 [0.08 0.36 0.92]
0.7 25 [0.07692308 0.38461538 0.88461538]
0.7 26 [0.11111111 0.40740741 0.88888889]
0.7 27 [0.10714286 0.42857143 0.89285714]
0.7 2